# 1. Import  Library

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import torch
import torch.nn as nn
import re

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, ArrayDictionary, StopWordRemover
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from lightgbm import LGBMClassifier
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel


# 2. Data loading

In [ ]:
df = pd.read_csv('reviews_mcd.csv')
df = pd.concat([df, df_2])
df.head()

,nama_tempat,user,review,rating
0,Unknown,Agustus Duapuluhtiga,Anak anak sangat suka karena ada playland soal...,5
1,Unknown,roy kurniawan,Paling suka ngopi panas pagi di MCD 😬\nKopi Hi...,5
2,Unknown,Mona,Pagi ini jam 5.45 pesan lewat drive thru judul...,1
3,Unknown,Bang Dammm,Makanannya enak banget! Pelayanannya ramah dan...,5
4,Unknown,Peggy Marcella,Baru beli mcd di raya darmo. Ini worst experie...,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5193 entries, 0 to 2145
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   nama_tempat  5193 non-null   object
 1   user         5188 non-null   object
 2   review       5180 non-null   object
 3   rating       5193 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 202.9+ KB


In [6]:
df.describe()

,rating
count,5193.000000
mean,4.377816
std,1.027803
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


In [7]:
df = df[['review', 'rating']]

# 3. Labeling

In [8]:
df['rating'] = df['rating'].astype(str)

df.loc[df['rating'].isin(['1', '2']), 'rating'] = 'negative'
df.loc[df['rating'] == '3', 'rating'] = 'netral'
df.loc[df['rating'].isin(['4', '5']), 'rating'] = 'positive'

In [9]:
df.head()

,review,rating
0,Anak anak sangat suka karena ada playland soal...,positive
1,Paling suka ngopi panas pagi di MCD 😬\nKopi Hi...,positive
2,Pagi ini jam 5.45 pesan lewat drive thru judul...,negative
3,Makanannya enak banget! Pelayanannya ramah dan...,positive
4,Baru beli mcd di raya darmo. Ini worst experie...,negative


In [10]:
df['rating'].value_counts()

rating
positive    4451
netral       395
negative     347
Name: count, dtype: int64

# 4. Cleanging text

In [11]:
df = df.dropna()

In [12]:
df['review'].duplicated().sum()

211

In [13]:
df = df.drop_duplicates(subset=['review'])
df['review'].duplicated().sum()

0

## pengahpusan karakter

In [14]:
def clean_text(teks):
    teks = teks.lower()
    
    # Menghapus URL
    teks = re.sub(r'https?://\S+|www\.\S+|bit\.ly/\S+', ' ', teks)

    # Menghapus @ dan #
    teks = re.sub(r'(?<=\s)#(?=\w)', ' ', teks)
    teks = re.sub(r'@\w+', ' ', teks)
    
    # Menghapus karakter non-alfabet
    teks = re.sub(r'[^a-z\s]', ' ', teks)
    
    # Menghapus kata satu huruf di mana pun posisinya
    teks = re.sub(r'\b[a-z]\b', ' ', teks)
    
    # Menghapus spasi berlebihan
    teks = re.sub(r'\s+', ' ', teks).strip()
    
    return teks

In [15]:
df['review'] = df['review'].apply(clean_text)

## normalisasi kata

In [16]:
# Normalisasi
norm = {'yg ': ' yang ', 
        'nggak ':' tidak ', 
        'gak ':' tidak ' ,
        'bangetdari ':' banget dari ',
        'vibes ':' suasana ' ,
        'mantab ':' keren ',
        'benarsetuju ': ' benar setuju ',
        'ganjarmahfud ':' ganjar mahfud ', 
        'stylish ':' bergaya ',
        'ngapusi ':' bohong ',
        'gede ':' besar ', 
        'all in ':' yakin ', 
        'blokkkkk ':' goblok ', 
        'blokkkk ':' goblok ', 
        'blokkk ':' goblok ' ,
        'blokk ':' goblok ' , 
        'blok ':' goblok ',
        'ri ':' republik indonesia' , 
        'tni':'tentara nasional indonesia',
        'polri':'polisi republik indonesia',
        'dpr':'dewan perwakilan rakyat',
        'kem3nangan ':' kemenangan ', 
        'sat set ':' cepat ' ,
        'ala ':' dari ' ,
        'best ':' terbaik ' ,
        'mantab ':' mantap ' ,
        'bgttt ' : ' banget ' ,
        "gue ": " saya ", 
        "hrs ": " harus ", 
        "fixed ":" tetap ", 
        'blom ':' belum ', 
        'aing ': ' aku ', 
        'tehnologi ':' teknologi ', 
        'jd ':' jadi ', 
        'dg ':' dengan ', 
        'kudu ':' harus ', 
        'jk ':' jika ', 
        'problem ':' masalah ', 
        'iru ':' itu ', 
        'duit ':' uang ', 
        'duid ':' uang ', 
        'bgsd ':' bangsat ', 
        'ngenes':'sengsara',
        'jt ':' juta ', 
        'stop ':' berhenti ', 
        'ngeri ':' seram ', 
        'turu ':' tidur ', 
        'early ':' awal ', 
        'pertamna ':' pertamina ', 
        'yg ':' yang ', 
        'mnurut ':' menurut ', 
        'trus ':' terus ', 
        'msh ':' masih ', 
        'simple ':' mudah ', 
        'worth ':' layak ', 
        'problem ':' masalah ', 
        'hny ':' hanya ', 
        'dn ':' dan ', 
        'jln ':' jalan ', 
        'bgt ':' banget ', 
        'yg ':' yang ', 
        'ga ': ' tidak ', 
        'text ':' teks ', 
        'end ':' selesai ', 
        'kelen ':' kalian ', 
        'jd ':' jadi ', 
        'tuk ':' untuk ', 
        'kk ':' kakak ',
        'punten':'permisi',
        'kunker':'kunjungan kerja',
        'ultah':'ulang tahun',
        'ajg':'anjing',
        'anjg':'anjing',
        'fav':'favorit',
        'mcd': 'mcdonalds',
        'mc':'mcdonalds',
        'eskrim': 'icecream',
        'kmrin':'kemarin',
        'cz':'cause',
        'mba':'mbak',
        'klau':'kalau',
        'gbr':'gambar'
        }

norm_cleaned = {}

# Iterasi setiap pasangan key-value
for key, value in norm.items():
    # Hapus spasi berlebih menggunakan .strip()
    cleaned_key = key.strip()
    cleaned_value = value.strip()
    
    # Masukkan ke kamus baru
    norm_cleaned[cleaned_key] = cleaned_value

norm = norm_cleaned

In [17]:
df_slang = pd.read_csv("https://raw.githubusercontent.com/refanz/indonesian-slangwords/master/data/indonesian-slangwords.csv")
df_norm = pd.DataFrame(list(norm.items()), columns=['@', 'di'])
df_slang = pd.concat([df_slang, df_norm], ignore_index=True)

In [18]:
slang = df_slang['@']
mean = df_slang['di']
dict_slang = dict(zip(slang, mean))

In [19]:
def normalisasi_kata(teks):    
    # 2. Tokenisasi: Memisahkan string menjadi daftar kata
    tokens = teks.split()
    
    # 3. Normalisasi: Ganti setiap kata sesuai kamus
    normalized_tokens = [dict_slang.get(token, token) for token in tokens]
    
    # 4. Penggabungan: Gabungkan kembali kata-kata menjadi string
    return ' '.join(normalized_tokens)

# Terapkan fungsi normalisasi ke kolom 'full_text'
df['review'] = df['review'].apply(normalisasi_kata)

## stopwords removal

In [20]:
more_stopwords = ['donalds', 'nya', 'kan']
stopwords = StopWordRemoverFactory().get_stop_words()
stopwords.extend(more_stopwords)

stopword_array = ArrayDictionary(stopwords)
stop_words_remover = StopWordRemover(stopword_array)


In [21]:
def stopwords_removal(teks):
    teks = stop_words_remover.remove(teks)
    return teks

df['review'] = df['review'].apply(lambda x: stopwords_removal(x))

In [22]:
df.to_csv('clean.csv', index=False)

## stemming

In [23]:
def stemming(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    
      # Pastikan input adalah string dan pecah menjadi kata-kata (tokenisasi)
    words = str(text).split()
    
    do = []
    for w in words:
        dt = stemmer.stem(w)
        do.append(dt)

    d_clean = " ".join(do)
    return d_clean

df.loc[:, 'review'] = df['review'].apply(stemming)

In [24]:
df.to_csv('clean.csv', index=False)

# Preprocessing

In [25]:
le = LabelEncoder()

In [26]:
df['label'] = le.fit_transform(df['rating'])

In [27]:
df.head()

,review,rating,label
0,anak anak sangat suka ada playland soal makan ...,positive,2
1,paling suka ngopi panas pagi mcdonalds kopi hi...,positive,2
2,pagi jam pesan lewat drive thru judul paket ce...,negative,0
3,makan enak banget layan ramah cepat tempat ber...,positive,2
4,baru beli mcdonalds raya darmo worst experienc...,negative,0


In [28]:
X = df['review']
y = df['label']

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [30]:
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Training

## LGBM

### tokenizer

In [31]:
vectorizer = TfidfVectorizer()

In [32]:
# X_train_transformed = vectorizer.fit_transform(X_train)

# X_test_transformed = vectorizer.transform(X_test)

X_train_transformed = pd.DataFrame(
    vectorizer.fit_transform(X_train).toarray(),
    columns=vectorizer.get_feature_names_out()
)

X_test_transformed = pd.DataFrame(
    vectorizer.transform(X_test).toarray(),
    columns=vectorizer.get_feature_names_out()
)

In [33]:
# Inisialisasi dan latih model LGBM
lgbm_model = LGBMClassifier()
lgbm_model.fit(X_train_transformed, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007895 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8552
[LightGBM] [Info] Number of data points in the train set: 3975, number of used features: 311
[LightGBM] [Info] Start training from score -2.663763
[LightGBM] [Info] Start training from score -2.554439
[LightGBM] [Info] Start training from score -0.159490


,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.1
,n_estimators,100
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [34]:
# Prediksi training
y_pred_train = lgbm_model.predict(X_train_transformed)
train_acc = accuracy_score(y_train, y_pred_train)
print(f"Akurasi training model: {train_acc:.2f}")

# Prediksi testing
y_pred_test = lgbm_model.predict(X_test_transformed)
test_acc = accuracy_score(y_test, y_pred_test)
print(f"Akurasi testing model: {test_acc:.2f}")

Akurasi training model: 0.98
Akurasi testing model: 0.86


## NN Layer

In [35]:
MODEL_NAME = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
bert_model = AutoModel.from_pretrained(MODEL_NAME)

In [36]:
class ReviewDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        enc = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt"
        )

        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long)
        }

In [ ]:
# # arsitektur lstm 1

# class BertLSTM(nn.Module):
#     def __init__(self, bert_model, hidden_dim=256, num_classes=3):
#         super(BertLSTM, self).__init__()
#         self.bert = bert_model
#         self.lstm = nn.LSTM(
#             input_size=self.bert.config.hidden_size,  
#             hidden_size=hidden_dim,
#             num_layers=2,
#             batch_first=True,
#             bidirectional=True
#         )
#         self.dropout = nn.Dropout(0.3)
#         self.fc = nn.Linear(hidden_dim * 2, num_classes)  
#         self.fc1 = nn.Linear(hidden_dim * 2, 128)
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(128, num_classes)

#     def forward(self, input_ids, attention_mask):
#         with torch.no_grad(): 
#             outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
#             sequence_output = outputs.last_hidden_state 

#         lstm_out, _ = self.lstm(sequence_output) 
#         pooled = torch.mean(lstm_out, dim=1)  
#         out = self.dropout(pooled)
#         logits = self.fc(out)
#         return logits

In [ ]:
# # arstitektur lstm 2

# class BertLSTM(nn.Module):
#     def __init__(self, bert_model, hidden_dim=256, num_classes=3):
#         super(BertLSTM, self).__init__()
#         self.bert = bert_model
#         self.lstm = nn.LSTM(
#             input_size=self.bert.config.hidden_size,  
#             hidden_size=hidden_dim,
#             num_layers=2,
#             batch_first=True,
#             bidirectional=True
#         )
        
#         # Dense layers stack (lebih dalam)
#         self.fc1 = nn.Linear(hidden_dim * 2, 256)   
#         self.bn1 = nn.BatchNorm1d(256)
#         self.fc2 = nn.Linear(256, 128)              
#         self.bn2 = nn.BatchNorm1d(128)
#         self.fc3 = nn.Linear(128, 64)               
#         self.bn3 = nn.BatchNorm1d(64)
#         self.fc_out = nn.Linear(64, num_classes)    
        
#         self.relu = nn.ReLU()
#         self.dropout = nn.Dropout(0.3)

#     def forward(self, input_ids, attention_mask):
#         with torch.no_grad():  
#             outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
#             sequence_output = outputs.last_hidden_state  #

#         lstm_out, _ = self.lstm(sequence_output)  
#         pooled = torch.mean(lstm_out, dim=1)      

#         # Dense layers stack
#         x = self.fc1(pooled)
#         x = self.bn1(x)
#         x = self.relu(x)
#         x = self.dropout(x)

#         x = self.fc2(x)
#         x = self.bn2(x)
#         x = self.relu(x)
#         x = self.dropout(x)

#         x = self.fc3(x)
#         x = self.bn3(x)
#         x = self.relu(x)
#         x = self.dropout(x)

#         logits = self.fc_out(x)
#         return logits


In [ ]:
# arsitektur lstm 3 (best performance)

class BertLSTM(nn.Module):
    def __init__(self, bert_model, hidden_dim=256, num_classes=3):
        super(BertLSTM, self).__init__()
        self.bert = bert_model
        self.lstm = nn.LSTM(
            input_size=self.bert.config.hidden_size,  
            hidden_size=hidden_dim,
            num_layers=2,
            batch_first=True,
            bidirectional=True
        )
        
        self.dropout = nn.Dropout(0.3)

        # Tambahan beberapa dense layer setelah LSTM
        self.fc1 = nn.Linear(hidden_dim * 2, 256)   
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(256, 128)              
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(128, 64)               
        self.relu3 = nn.ReLU()
        self.fc_out = nn.Linear(64, num_classes)    

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():  
            outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
            sequence_output = outputs.last_hidden_state  

        lstm_out, _ = self.lstm(sequence_output)         
        pooled = torch.mean(lstm_out, dim=1)             

        # Dense layers stack
        x = self.fc1(pooled)
        x = self.relu1(x)
        x = self.dropout(x)

        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout(x)

        x = self.fc3(x)
        x = self.relu3(x)
        x = self.dropout(x)

        logits = self.fc_out(x)
        return logits


In [ ]:
# # arsitektur lstm 4

# class BertLSTM(nn.Module):
#     def __init__(self, bert_model, hidden_dim=256, num_classes=3):
#         super(BertLSTM, self).__init__()
#         self.bert = bert_model
#         self.lstm = nn.LSTM(
#             input_size=self.bert.config.hidden_size,  
#             hidden_size=hidden_dim,
#             num_layers=2,
#             batch_first=True,
#             bidirectional=True,
#             dropout=0.3  # dropout antar LSTM layers
#         )
        
#         # Dense layers dengan BatchNorm + Dropout bervariasi
#         self.fc1 = nn.Linear(hidden_dim * 2, 256)   
#         self.bn1 = nn.BatchNorm1d(256)
#         self.relu1 = nn.ReLU()
#         self.drop1 = nn.Dropout(0.5)

#         self.fc2 = nn.Linear(256, 128)              
#         self.bn2 = nn.BatchNorm1d(128)
#         self.relu2 = nn.ReLU()
#         self.drop2 = nn.Dropout(0.4)

#         self.fc3 = nn.Linear(128, 64)               
#         self.bn3 = nn.BatchNorm1d(64)
#         self.relu3 = nn.ReLU()
#         self.drop3 = nn.Dropout(0.3)

#         self.fc_out = nn.Linear(64, num_classes)    

#     def forward(self, input_ids, attention_mask):
#         with torch.no_grad():  
#             outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
#             sequence_output = outputs.last_hidden_state  

#         lstm_out, _ = self.lstm(sequence_output)         
#         pooled = torch.mean(lstm_out, dim=1)             

#         # Dense stack dengan BN + Dropout
#         x = self.fc1(pooled)
#         x = self.bn1(x)
#         x = self.relu1(x)
#         x = self.drop1(x)

#         x = self.fc2(x)
#         x = self.bn2(x)
#         x = self.relu2(x)
#         x = self.drop2(x)

#         x = self.fc3(x)
#         x = self.bn3(x)
#         x = self.relu3(x)
#         x = self.drop3(x)

#         logits = self.fc_out(x)
#         return logits


In [ ]:
# arsitektur GRU 1

class BertGRU(nn.Module):
    def __init__(self, bert_model, hidden_dim=256, num_classes=3):
        super(BertGRU, self).__init__()
        self.bert = bert_model
        self.gru = nn.GRU(
            input_size=self.bert.config.hidden_size,  
            hidden_size=hidden_dim,
            num_layers=2,
            batch_first=True,
            bidirectional=True,
            dropout=0.3  
        )
        
        # Dense layers dengan BatchNorm + Dropout
        self.fc1 = nn.Linear(hidden_dim * 2, 256)  
        self.bn1 = nn.BatchNorm1d(256)
        self.relu1 = nn.ReLU()
        self.drop1 = nn.Dropout(0.5)

        self.fc2 = nn.Linear(256, 128)              
        self.bn2 = nn.BatchNorm1d(128)
        self.relu2 = nn.ReLU()
        self.drop2 = nn.Dropout(0.4)

        self.fc3 = nn.Linear(128, 64)               
        self.bn3 = nn.BatchNorm1d(64)
        self.relu3 = nn.ReLU()
        self.drop3 = nn.Dropout(0.3)

        self.fc_out = nn.Linear(64, num_classes)    

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():  
            outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
            sequence_output = outputs.last_hidden_state  

        gru_out, _ = self.gru(sequence_output)           
        pooled = torch.mean(gru_out, dim=1)              

        # Dense stack dengan BN + Dropout
        x = self.fc1(pooled)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.drop1(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.drop2(x)

        x = self.fc3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.drop3(x)

        logits = self.fc_out(x)
        return logits


In [42]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

device

device(type='mps')

### LSTM

In [ ]:
train_dataset = ReviewDataset(X_train, y_train, tokenizer)
test_dataset = ReviewDataset(X_test, y_test, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

model = BertLSTM(bert_model, hidden_dim=128, num_classes=len(set(y_train)))
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [44]:
EPOCHS = 6
for epoch in range(EPOCHS):
    model.train()
    total_loss, total_preds, total_labels = 0, [], []

    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        total_preds.extend(preds)
        total_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(total_labels, total_preds)
    print(f"Epoch {epoch+1}/{EPOCHS} | Loss={total_loss/len(train_loader):.4f} | Train Acc={acc:.4f}")

Epoch 1/6 | Loss=0.5324 | Train Acc=0.8372
Epoch 2/6 | Loss=0.4774 | Train Acc=0.8526
Epoch 3/6 | Loss=0.4708 | Train Acc=0.8528
Epoch 4/6 | Loss=0.4588 | Train Acc=0.8511
Epoch 5/6 | Loss=0.4493 | Train Acc=0.8518
Epoch 6/6 | Loss=0.4462 | Train Acc=0.8531


In [229]:
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()

        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

test_acc = accuracy_score(all_labels, all_preds)
print(f"\nTest Accuracy: {test_acc:.4f}")
print("\nClassification Report:\n", classification_report(all_labels, all_preds, digits=4))


Test Accuracy: 0.8669

Classification Report:
               precision    recall  f1-score   support

           0     0.5484    0.4359    0.4857        39
           1     0.0000    0.0000    0.0000        50
           2     0.8838    0.9810    0.9299       527

    accuracy                         0.8669       616
   macro avg     0.4774    0.4723    0.4719       616
weighted avg     0.7908    0.8669    0.8263       616



/opt/anaconda3/envs/ml_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/ml_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/ml_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result

Hasil model menggunakan model Bert + LSTM dengan layer nomer 3(best performance).
mempunyai akurasi pada testing sebesar 0.8669

## GRU

In [ ]:
train_dataset = ReviewDataset(X_train, y_train, tokenizer)
test_dataset = ReviewDataset(X_test, y_test, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

model = BertGRU(bert_model, hidden_dim=128, num_classes=len(set(y_train)))
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [46]:
EPOCHS = 6
for epoch in range(EPOCHS):
    model.train()
    total_loss, total_preds, total_labels = 0, [], []

    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        total_preds.extend(preds)
        total_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(total_labels, total_preds)
    print(f"Epoch {epoch+1}/{EPOCHS} | Loss={total_loss/len(train_loader):.4f} | Train Acc={acc:.4f}")

Epoch 1/6 | Loss=0.5611 | Train Acc=0.8282
Epoch 2/6 | Loss=0.4819 | Train Acc=0.8516
Epoch 3/6 | Loss=0.4667 | Train Acc=0.8526
Epoch 4/6 | Loss=0.4590 | Train Acc=0.8518
Epoch 5/6 | Loss=0.4535 | Train Acc=0.8526
Epoch 6/6 | Loss=0.4374 | Train Acc=0.8548


In [48]:
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()

        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

test_acc = accuracy_score(all_labels, all_preds)
print(f"\nTest Accuracy: {test_acc:.4f}")
print("\nClassification Report:\n", classification_report(all_labels, all_preds, digits=4))


Test Accuracy: 0.8622

Classification Report:
               precision    recall  f1-score   support

           0     0.5088    0.4203    0.4603        69
           1     0.0000    0.0000    0.0000        78
           2     0.8837    0.9776    0.9283       847

    accuracy                         0.8622       994
   macro avg     0.4641    0.4660    0.4629       994
weighted avg     0.7883    0.8622    0.8229       994



/opt/anaconda3/envs/ml_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/ml_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/ml_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result